Author: Sean Tulin
<br>
PHYS 2030

# <font color=#46769B>Lesson 8: Second-order methods (and beyond)</font>

## <font color=#46769B>Introduction</font>

Let's continue the discussion of __initial value problems__ (IVPs). We introduced Euler's method, the simplest algorithm for solving IVPs but also the least accurate. As a first-order method, the global error scales proportional to $h$, or with $1/N$, where $N$ is the number of steps over a fixed interval.

Here we consider __second-order methods__. The idea is that each step is accurate including terms *quadratic* in $h$. Now, $\mathcal{O}(h^3)$ terms are neglected, and therefore the local error (per step) is proportional to $h^3$ and the global error is proportional to $h^2$. 

Higher-order methods can offer a dramatic improvement in the performance of the algorithm. Suppose we require $N=10^6$ steps to achieve a certain level of accuracy with Euler's method. For a second-order method, we only require $N=10^3$ steps to achieve a similar accuracy. 

## <font color=#46769B>Midpoint method</font>

A straightforward second-order generalization of Euler's method is the __midpoint method__. Let's consider a single ODE

$$y'(t) = f(t,y) \, .$$

The forward Euler method is based on a Taylor expansion

$$y(t+h) = y(t) + y'(t) \, h + \mathcal{O}(h^2) = y(t) + f(t,y) \, h + \mathcal{O}(h^2) \, , \qquad (1)$$

keeping only terms first-order in $h$. This allowed us to construct an algorithm to determine our solution iteratively one step at a time, evaluating the ($i+1$)-th value in terms of the $i$th one: 

$$y_{i+1} = y_i + f(t_i,y_i) \, h \, .$$

Next, let's consider the Taylor expansion of $y(t+h)$ around the "midpoint" $t+\tfrac{h}{2}$:

$$y(t+h) = y\left(t+\tfrac{h}{2}\right) + y'\left(t+\tfrac{h}{2}\right) \, \frac{h}{2} +  \frac{1}{2} y''\left(t+\tfrac{h}{2}\right) \left( \frac{h}{2} \right)^2 + \mathcal{O}(h^3) \, .$$

Let us also expand $y(t)$ around the same point, $t+\tfrac{h}{2}$.

$$y(t) = y\left(t+\tfrac{h}{2}\right) - y'\left(t+\tfrac{h}{2}\right) \, \frac{h}{2} +  \frac{1}{2} y''\left(t+\tfrac{h}{2}\right) \left( \frac{h}{2} \right)^2 + \mathcal{O}(h^3) \, .$$

Subtracting these equations, we have

$$ y(t+h) - y(t) = y'\left(t+\tfrac{h}{2}\right) \, h + \mathcal{O}(h^3) 
= f\left(t+\tfrac{h}{2}, y\left(t+\tfrac{h}{2}\right)\right) \, h\, + \mathcal{O}(h^3) . \qquad (2)$$

That is, the $\mathcal{O}(h^2)$ terms cancel out, and our result is now accurate up to $\mathcal{O}(h^3)$ terms. Next, we need to evaluate $y\left(t+\tfrac{h}{2}\right)$ that appears in $f$. We perform another Taylor expansion: the same as in Eq. (1) but with $h \to \tfrac{h}{2}$:

$$y\left(t+\tfrac{h}{2}\right) = y(t) + y'(t) \, \frac{h}{2} + \mathcal{O}(h^2) = y(t) + f(t,y) \, \frac{h}{2} + \mathcal{O}(h^2) \, . \qquad (3)$$

Plugging in, we have:

$$y(t+h) - y(t) = f\left(t+\tfrac{h}{2}, y(t)+ \tfrac{h}{2} f(t,y(t)) \right) \, h + \mathcal{O}(h^3)\, \qquad (4)$$

Note that Eq. (3) has terms arising at $\mathcal{O}(h^2)$, but the right-hand side of Eq. (2) is already order $h$. So, plugging in Eq. (3) is accurate up to $\mathcal{O}(h^3)$, as we want.

Now, we can convert Eq. (4) into a numerical algorithm for solving our ODE, one step at a time. We have

$$y_{i+1} = y_i + f\left(t_i+\tfrac{h}{2}, y_i + \tfrac{h}{2} f(t_i,y_i) \right) h \, .$$

This is an explicit method, where the right-hand side depends on quantities that are calculated given $y_i$ at the previous step $t_i$.

Note that to compute the next step, $y_{i+1}$, we need to evaluate the function $f$ *twice*, in two *stages*:
- First, we evaluate $f(t_i,y_i)$, as in Euler's method (first stage). With this, we evaluate $y$ at the midpoint:

$$t_{i+1/2} = t_i + \tfrac{h}{2} \, , \qquad y_{i+1/2} = y_i + \tfrac{h}{2} f(t_i,y_i)\, . $$

- Second, we evaluate $f(t_{i+1/2},y_{i+1/2})$ at the midpoint (second stage). With this, we evaluate $y_{i+1}$ according to Eq. (4).

This is a general feature of *all* numerical algorithms for solving initial value problems. If we have a method that is accurate at order $s$, then we require at least $k$ stages and therefore $k$ evaluations of the function $f(t,y)$ per step. Even with these extra evaluations, it is still a big improvement to work with higher-order methods. For example:
- For Euler's method ($s=1$), say we perform $N=10^6$ steps to achieve a given accuracy. The time of the algorithm is determined by the fact that we must do $10^6$ evaluations of $f(t,y)$.
- For the midpoint method ($s=2$), the similar accuracy is achieved for $N=10^3$ steps, and we must do $2N = 2000$ evaluations of $f(t,y)$.
- For a fourth-order method ($s=4$), a similar accuracy is achieved for $N=10^{1.5} \approx 30$ steps, and we must do $4N \approx 120$ evaluations of $f(t,y)$. 

In the case of a system of coupled ODEs, all the arguments described above follow similarly. The midpoint method algorithm for determining the next step is generalized to a *vector* equation

$$\mathbf{y}_{i+1} = \mathbf{y}_i + \mathbf{f}\left(t_i+\tfrac{h}{2}, \mathbf{y}_i + \tfrac{h}{2} \mathbf{f}(t_i,\mathbf{y}_i) \right) h \, .$$


## <font color=#46769B>Runge-Kutta methods</font>

Euler's method and the midpoint method are part of a general family of algorithms known as __Runge-Kutta (RK) methods__ for solving IVPs. Euler's method is the unique first-order RK algorithm with one stage (RK1). At second order, there is a family of two-stage algorithms (RK2), of which the midpoint method is one example. In practice, the algorithms that are widely used in numerical solutions to IVPs are four-stage algorithms (RK4), which are fourth-order methods. A variant of RK4 is used in `scipy.integrate.solve_ivp`. There are videos online describing [RK2](https://www.youtube.com/watch?v=ihEfPp5WRcE) and [RK4](https://www.youtube.com/watch?v=hGN54bkE8Ac) that are highly recommended.

### <font color=#46769B>Derivation of second-order method</font>

Here we derive the general formulation of RK2. Consider the Taylor expansion

$$y(t+h) = y(t) + y'(t) \, h + \frac{1}{2} y''(t)\, h^2 + \mathcal{O}(h^3) \, , \qquad (5)$$

where we retain terms up to second order in $h$. The second derivative term is evaluated as follows

$$y''(t) = \frac{d}{dt} f(t,y(t)) = \frac{\partial f}{\partial t} + \frac{\partial f}{\partial y} \frac{dy}{dt} 
= \frac{\partial f}{\partial t} + \frac{\partial f}{\partial y} f(t,y) \, .$$

We are taking the *total derivative* of $f(t,y)$ with respect to $t$, so we need to account for the fact that $f(t,y)$ depends on $t$ explicitly through the first argument and implicitly through the second argument $y(t)$.

Now, in principle, we are done. If we can calculate derivatives of $f(t,y)$ analytically, we can use those to compute $y''(t)$ (and if we want to go beyond second order, we could calculate higher derivatives as well). On the other hand, it is very useful to consider an alternative approach where we *do not* need to evaluate derivatives of $f(t,y)$. For some problems, it may be impossible or impractical to compute them. For example, for a system of coupled ODEs, one would need to consider all possible derivatives of *each* $f_i$ with respect to *every* function $y_j$.

Here we will show that the second derivative term, $y''(t)$, can be calculated in terms of the function $f$ without evaluating derivatives of $f$. Mathematically, we can consider $f(t,y)$ as a function of two variables, $t$ and $y$, that we can adjust independently. 
Suppose we shift $t \to t + \Delta t$ and $y \to y + \Delta y$. How is $f(t + \Delta t, y + \Delta y)$ related to $f(t, y)$? We can perform a multivariate Taylor expansion

$$f(t + \Delta t, y + \Delta y) = f(t,y) + \frac{\partial f}{\partial t} \, \Delta t + \frac{\partial f}{\partial y} \, \Delta y + \mathcal{O}(\Delta t^2, \Delta y^2, \Delta t \Delta y) \, . \qquad (6) $$

We wish to consider a shift $\Delta t = \alpha h$, where $\alpha$ is a free parameter in the range $0 < \alpha \le 1$. (Ultimately, we will set $t \to t_i$, so we do not wish to consider a shift larger than the step-size $h$.) We are free to consider any $\Delta y$ we like. Let's take

$$\Delta y = \alpha h f(t,y) \, .$$

With this choice, Eq. (6) becomes

$$f\left(t + \alpha h, y + \alpha h f(t,y)\right) = f(t,y) + \, \alpha h \left( \frac{\partial f}{\partial t} + \frac{\partial f}{\partial y} \, f(t,y) \right)  + \mathcal{O}(h^2) \, .$$

and the term in parantheses is just $y''(t)$. We can solve for $y''(t)$:

$$y''(t) = \frac{1}{\alpha h} \Big( f\big(t + \alpha h, y + \alpha h f(t,y)\big) - f(t,y) \Big) + \mathcal{O}(h)$$

Now, we can plug into our original Taylor expansion in Eq. (5) to get

$$ y(t+h) = y(t) + f(t,y) \, h + \frac{h}{2\alpha}  \Big( f\big(t + \alpha h, y + \alpha h f(t,y)\big) - f(t,y) \Big) + \mathcal{O}(h^3) $$

Rearranging terms, we have

$$ y(t+h) = y(t) + h \left(1 - \frac{1}{2\alpha}\right) f(t,y)  + \frac{h}{2\alpha}  f\big(t + \alpha h, y + \alpha h f(t,y)\big) + \mathcal{O}(h^3) \, .$$

This formula allows us to write down our algorithm for computing the next step from the previous one. Dropping the $\mathcal{O}(h^3)$ terms, we have

$$y_{i+1} = y_i + h \left( \left(1 - \frac{1}{2\alpha}\right) f(t_i,y_i) + \frac{1}{2\alpha}  f\big(t_i + \alpha h, y_i + \alpha h f(t_i,y_i)\big) \right) \, .$$

We are free to choose $\alpha$ as we like. Taking $\alpha=\tfrac{1}{2}$ gives us the midpoint method described above. Taking $\alpha=1$ is known as __Heun's method__. That is:

$$y_{i+1} = y_i + h \times \left\{ \begin{array}{lc} f\left(t_i + \tfrac{h}{2}, y_i + \tfrac{h}{2} f(t_i,y_i) \right) & {\rm midpoint \; method} \\
\tfrac{1}{2} \left( f(t_i,y_i) + f(t_i+h,y_i+h f(t_i,y_i)) \right) & {\rm Heun's \; method} \end{array} \right.
$$


## <font color=#46769B>Higher-order RK algorithms</font>

When it comes to solving IVPs numerically, the classic algorithm widely used in practice is __RK4__. This method is accurate up to fourth-order in the step-size $h$, which means that the local error is $\mathcal{O}(h^5)$ and the global error is $\mathcal{O}(h^4)$. Once you get the hang of RK2, it is not too much extra effort implement RK4 . 

We will not attempt to *prove* that RK4 is as accurate as advertised, as we did for RK2. The proof is quite tedious and can be found [online](https://math.stackexchange.com/questions/2636121/prove-that-runge-kutta-method-rk4-is-of-order-4).

Let's describe the general algorithm for explicit RK methods of order $s$. First, we will recap Euler's method ($s=1$) and our second-order methods ($s=2$) in a different notation that we can generalize to higher order. The basic formula for updating the next value is

$$y_{i+1} = y_i + h k $$

where $k$ is simply the slope. The slope $k$ is the thing that needs to be calculated to obtain the next value, and higher-order algorithms will give us better estimates for $k$ so that our next value is accurate up to local errors at $\mathcal{O}(h^{s+1})$.

The RK algorithm will calculate the slope $k$ in $s$ stages, each time refining our estimate for $k$.

- *Euler's method (one-stage algorithm):* We calculate the slope only once:
$$\begin{array}{ll} k_1 = f(t_i,y_i) & {\rm stage \; 1} \end{array} \, $$ 
Finally, we take $k=k_1$ and we are done.


- *Mid-point method (two-stage algorithm):* We calculate the slope twice:
$$\begin{array}{ll} k_1 = f(t_i,y_i) & {\rm stage \; 1} \\  
k_2 = f(t_i + \tfrac{h}{2}, y_i + \tfrac{h}{2} k_1) & {\rm stage \; 2} \end{array} \, .$$
Finally, we set $k = k_2$.
    

- *Heun's method (two-stage algorithm):* We calculate the slope twice:
$$\begin{array}{ll} k_1 = f(t_i,y_i) & {\rm stage \; 1} \\  
k_2 = f(t_i + h, y_i + h k_1) & {\rm stage \; 2} \end{array} \, .$$
Finally, we take the mean of both slopes, $k = \tfrac{1}{2} \left( k_1 + k_2 \right)$.


- *Classic RK4 (four-stage algorithm):* Similar to RK2, there is no unique prescription for RK4. The method described here is the one most commonly used. We calculate four slopes:
$$\begin{array}{ll} k_1 = f(t_i,y_i) & {\rm stage \; 1} \\  
k_2 = f(t_i + \tfrac{h}{2}, y_i + \tfrac{h}{2} k_1) & {\rm stage \; 2} \\ 
k_3 = f(t_i + \tfrac{h}{2}, y_i + \tfrac{h}{2} k_2) & {\rm stage \; 3} \\  
k_4 = f(t_i + h, y_i + h k_3) & {\rm stage \; 4} \end{array} \, .$$
Finally, we take the following linear combination: 
$$k = \tfrac{1}{6} \left( k_1 + 2k_2 + 2 k_3 + k_4 \right) \, .$$

RK methods are easily generalized to systems of coupled ODEs. The next value is computed as

$$\mathbf{y}_{i+1} = \mathbf{y}_i + h \, \mathbf{k}$$

where now the slope $\mathbf{k}$ is a vector like $\mathbf{y}$. All the above formulas apply, with the replacements $y \to \mathbf{y}$ and $k \to \mathbf{k}$. (That is, each slope $\mathbf{k}_1$, $\mathbf{k}_2$, etc. is a vector.)

